# 0. Install Dependencies

# 1. Test Random Environment with OpenAI Gym

In [1]:
from DumbGame import DumbGameEnv
env = DumbGameEnv()
states = env.observation_space.shape
actions = env.action_space.n
print(f"States:{states} Actions:{actions}")

States:(1,) Actions:5


# 2. Create a Deep Learning Model with Keras

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
states = env.observation_space.shape
actions = env.action_space.n

In [4]:
actions

5

In [5]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [12]:
del model 

In [14]:
model = build_model(states, actions)

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 125       
Total params: 773
Trainable params: 773
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [9]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [10]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [16]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 9:10 - reward: -1.0000

/Users/Ben/Home/Working/Hanabi/envs/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 73s 7ms/step - reward: -0.5974
402 episodes - episode_reward: -14.821 [-47.000, 5.000] - loss: 1.242 - mae: 5.141 - mean_q: -6.008

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: -0.5944
406 episodes - episode_reward: -14.675 [-52.000, 3.000] - loss: 1.479 - mae: 6.070 - mean_q: -7.153

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -0.5978
402 episodes - episode_reward: -14.876 [-59.000, 2.000] - loss: 1.487 - mae: 6.079 - mean_q: -7.167

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 71s 7ms/step - reward: -0.5982
402 episodes - episode_reward: -14.883 [-60.000, 4.000] - loss: 1.505 - mae: 6.153 - mean_q: -7.265

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 73s 7ms/step - reward: -0.6216
done, took 357.885 seconds


In [17]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...


KeyboardInterrupt: 

In [29]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


# 4. Reloading Agent from Memory

In [30]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [31]:
del model
del dqn
del env

In [9]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [10]:
dqn.load_weights('dqn_weights.h5f')

In [11]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
